DRAWSINESS, YAWN 

In [5]:
from cvzone import putTextRect, cornerRect
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot
from cvzone.SerialModule import SerialObject
from cvzone.FPS import FPS
import cv2
import time 
from datetime import datetime
import pandas as pd 


#Log File Info 
log_file_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\Drawsiness Database\Drawsiness_Log.xlsx'
log_df = pd.read_excel(log_file_path)

#Function for logging information 
def log_event(state):
        
        global log_df
    
        current_time = datetime.now()
        date_str = current_time.strftime("%Y-%m-%d")
        time_str = current_time.strftime("%H:%M:%S")
    
        # Create a new row to append
        new_entry = pd.DataFrame({'Date': [date_str], 'Time': [time_str], 'State': [state]})
        
        # Append the new row to the log DataFrame
        log_df = pd.concat([log_df, new_entry], ignore_index=True)
        
        # Write the updated DataFrame back to the Excel file
        log_df.to_excel(log_file_path, index=False)


# Initialize the webcam
# '2' indicates the third camera connected to the computer, '0' would usually refer to the built-in webcam
cap = cv2.VideoCapture(0)

arduino = SerialObject("COM7", digits=1) 
fpsReader = FPS(avgCount=30)

# Initialize FaceMeshDetector object
# staticMode: If True, the detection happens only once, else every frame
# maxFaces: Maximum number of faces to detect
# minDetectionCon: Minimum detection confidence threshold
# minTrackCon: Minimum tracking confidence threshold


detector = FaceMeshDetector(staticMode=False, maxFaces=1, minDetectionCon=0.5, minTrackCon=0.5)
plotY = LivePlot(640, 360, [20, 90])
plotX = LivePlot(640, 360, [20, 50],char='X')

#idList = [22, 23, 24, 26, 118, 157, 158, 159, 160, 161, 130, 243]
idList = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243,     0, 17, 61, 409]
idListLip = [0, 17, 61, 409]
ratioList=[] 
ratioListLip = []
blinkCounter = 0
flag = 0
color = (0, 0, 255)
color1 = (0, 255, 0)

#Drawsiness Counters

drawsinessCount=0
drawsinessFlag=True 

yawnFlag = True 
yawnCount = 0

flagLip = 0

# Start the loop to continually get frames from the webcam
while True:
    # Read the current frame from the webcam
    # success: Boolean, whether the frame was successfully grabbed
    # img: The current frame
    success, img = cap.read()
    H,W,_ = img.shape

    #roix1, roiy1, roix2, roiy2 = int(W*0.35), int(H*0.01), int(W-0.35*W), int(H-0.5*H)
    roix1, roiy1, roix2, roiy2 = int(W*0.35), int(H*0.05), int(W-0.35*W), int(H-0.4*H)
    imgroi = img[roiy1:roiy2, roix1:roix2]
    cornerRect(img, [roix1, roiy1, roix2-roix1, roiy2-roiy1],rt=2,colorR=(0,0,0),colorC=(255,0,0))
    #cv2.imshow("Check", img[roiy1:roiy2+5,roix1-5:roix2+5])

    # Find face mesh in the image
    # img: Updated image with the face mesh if draw=True
    # faces: Detected face information
    imgroi, faces = detector.findFaceMesh(imgroi, draw=False)
    

    # Check if any faces are detected
    if faces:
            face = faces[0]

            for id in idList: 
                  cv2.circle(img, (face[id][0]+roix1, face[id][1]+roiy1), 2, color1, cv2.FILLED)
            # Get specific points for the eye
            # leftEyeUpPoint: Point above the left eye
            # leftEyeDownPoint: Point below the left eye
            leftEyeUpPoint = face[159]
            leftEyeDownPoint = face[23] #23

            leftEyeLeftPoint = face[130]
            leftEyeRightPoint = face[243]

            lipTop= face[0]
            lipBottom = face[17]
            lipLeft = face [61]
            lipRight = face [409]

            # Assume P1, P2, P3, P4 are defined as (x, y) tuples
            points = [leftEyeUpPoint, leftEyeDownPoint, leftEyeLeftPoint, leftEyeRightPoint]
            adjusted_points = []

            pointsLip = [lipTop, lipBottom, lipLeft, lipRight]
            adjusted_points_lips = []

            for point in points:
                x, y = point
                x_adjusted = x + roix1
                y_adjusted = y + roiy1
                adjusted_points.append((x_adjusted, y_adjusted))

            leftEyeUpPoint, leftEyeDownPoint, leftEyeLeftPoint, leftEyeRightPoint = adjusted_points

            for pointLip in pointsLip:
                x, y = pointLip
                x_adjusted = x + roix1
                y_adjusted = y + roiy1
                adjusted_points_lips.append((x_adjusted, y_adjusted))

            lipTop, lipBottom, lipLeft, lipRight = adjusted_points_lips

            # cv2.line(img, leftEyeDownPoint,leftEyeUpPoint, (255,0,0),2)
            # cv2.line(img, leftEyeLeftPoint,leftEyeRightPoint, (0,255,0),2)

            #print(face)

            # Calculate the vertical distance between the eye points
            # leftEyeVerticalDistance: Distance between points above and below the left eye
            # info: Additional information (like coordinates)
            leftEyeVerticalDistance, info = detector.findDistance(leftEyeUpPoint, leftEyeDownPoint)
            leftEyeHorizontalDistance, info = detector.findDistance(leftEyeLeftPoint, leftEyeRightPoint)
            ratio = (leftEyeVerticalDistance/leftEyeHorizontalDistance)*100

            lipVerticalDistance, info = detector.findDistance(lipTop, lipBottom)
            lipHorizontalDistance, info = detector.findDistance(lipLeft, lipRight)
            ratioLip = (lipVerticalDistance/lipHorizontalDistance)*100



            ratioList.append(ratio)
            if len(ratioList)>5: 
                 ratioList.pop(0)

            ratioAvg = sum(ratioList)/len(ratioList)


            ratioListLip.append(ratioLip)
            if len(ratioListLip)>5: 
                 ratioListLip.pop(0)

            ratioAvgLip = sum(ratioListLip)/len(ratioListLip)

            if ratioAvg < 33: #33 #30
                 flag+=1
                 #imgPlot = plotY.update(ratioAvg, color= (0,255,0))
                 color1 = (0, 0, 255)

                 if flag >20:
                    putTextRect(img,"Drowsiness",(roix1+28, roiy2+60), scale=1.5, colorR= (0,0,210), colorB = (0,0,0), offset=8, thickness=2,border=3) 
                    arduino.sendData([1])
                    if drawsinessFlag: 
                        drawsinessCount+=1 
                        log_event("Sleep")
                    drawsinessFlag= False
            else: 
                 flag=0
                 drawsinessFlag = True 
                 color1 = (0, 255, 0)
            putTextRect(img,f'Drowsiness Count: {drawsinessCount}',(20, roiy2+100), scale=1.5, colorR= (230,0,0), colorB = (0,0,0), offset=8, thickness=2, border=3)


            if ratioAvgLip > 65: 
                 flagLip+=1
                 color2 = (0, 0, 255)

                 if flagLip >15:
                    putTextRect(img,"Yawn",(roix1+65, roiy2+30), scale=1.5, colorR= (0,0,210), colorB = (0,0,0), offset=8, thickness=2,border=3) 
                    arduino.sendData([1])
                    if yawnFlag: 
                        yawnCount+=1 
                        log_event("Yawn")
                    yawnFlag= False
            else: 
                 flagLip=0
                 yawnFlag = True 
                 color2 = (0, 255, 0)
            #putTextRect(img,f'Yawn Count: {yawnCount}',(20, roiy2+200), scale=1.5, colorR= (230,0,0), colorB = (0,0,0), offset=8, thickness=2, border=3)
            putTextRect(img,f'Yawn Count: {yawnCount}',(20, roiy2+150), scale=1.5, colorR= (230,0,0), colorB = (0,0,0), offset=8, thickness=2, border=3)

           

            # if flag!=0:
            #      flag+=1
            #      if flag>50: #10
            #           flag=0
            #           color = (0,0,255)


            imgPlot = plotX.update(ratioAvg, color1)
            imgPlotLip = plotY.update(ratioAvgLip, color2)
            cv2.imshow("Blink Plot", imgPlot)
            cv2.imshow("Lip Plot", imgPlotLip)

    else:
        # Reset everything when no face is detected
        blinkCounter = 0
        drawsinessCount=0
        yawnCount=0
       
    
    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(20, 50),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    # Display the image in a window named 'Image'
    cv2.imshow("Image", img)

    # Wait for 1 millisecond to check for any user input, keeping the window open
    if cv2.waitKey(1) & 0xFF == ord('q'):
       break
cap.release()
cv2.destroyAllWindows()


Attempt 1 of 5 to connect...
Attempt 1 failed. Retrying...
Attempt 2 of 5 to connect...
Attempt 2 failed. Retrying...
Attempt 3 of 5 to connect...
Attempt 3 failed. Retrying...
Attempt 4 of 5 to connect...
Attempt 4 failed. Retrying...
Attempt 5 of 5 to connect...
Attempt 5 failed. Retrying...
